In [ ]:
import pandas as pd
import json
import os
import numpy as np

with open(r'C:\Users\acer\Desktop\Ds\Data 2018-2023\Project\2023\202300222.json') as fd:
    sc = json.load(fd)


In [ ]:
# # Set option to display all rows
# pd.set_option('display.max_rows', None)

# # Set option to display full content in each cell without truncation
# pd.set_option('display.max_colwidth', None)

# # Optionally, set the display width to accommodate wide columns
# pd.set_option('display.width', None)

In [ ]:
sc

In [ ]:
au_list = []

# Navigate to 'author-group' and ensure it's a list
author_groups = sc.get('abstracts-retrieval-response', {}) \
                  .get('item', {}) \
                  .get('bibrecord', {}) \
                  .get('head', {}) \
                  .get('author-group', [])

if isinstance(author_groups, dict):
    author_groups = [author_groups]

for group in author_groups:
    authors = group.get('author', [])
    if not authors:
        continue  # Skip if there are no authors
    
    affiliation = group.get('affiliation', {})
    if not affiliation:
        continue  # Skip if affiliation is missing
    
    aff_id = affiliation.get('@afid', '')
    
    # Extract organization names
    org = affiliation.get('organization', [])
    if isinstance(org, dict):
        aff_name = org.get('$', '')
    elif isinstance(org, list):
        aff_name = ', '.join(item.get('$', '') for item in org)
    else:
        aff_name = ''
    
    aff_city = affiliation.get('affiliation-city', affiliation.get('city', ''))
    aff_country = affiliation.get('affiliation-country', affiliation.get('country', ''))
    
    for author in authors:
        indexed_name = author.get('ce:indexed-name', 'Unknown')
        au_list.append((indexed_name, aff_id, aff_name, aff_city, aff_country))

In [ ]:
au_list

In [ ]:
df_authors = pd.DataFrame(au_list, columns=['name', 'id', 'organization', 'city', 'country'])

In [ ]:
df_authors

In [ ]:
references = sc['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference']

ref_list = []
for ref in references:
    ref_id = ref.get('@id', '')
    fulltext = ref.get('ref-fulltext', '')
    publication_year = ref.get('ref-info', {}).get('ref-publicationyear', {}).get('@first', '')
    title = ref.get('ref-info', {}).get('ref-title', {}).get('ref-titletext', '')
    source_title = ref.get('ref-info', {}).get('ref-sourcetitle', '')
    
    authors = ref.get('ref-info', {}).get('ref-authors', {}).get('author', [])
    author_names = [author.get('ce:indexed-name', '') for author in authors]
    authors_str = ', '.join(author_names)
    
    ref_list.append({
        'id': ref_id,
        'fulltext': fulltext,
        'year': publication_year,
        'title': title,
        'source_title': source_title,
        'authors': authors_str
    })

In [ ]:
ref_list

In [ ]:
ref_df = pd.DataFrame(ref_list)

In [ ]:
ref_df

In [ ]:
import os
import json
import pandas as pd

folder_path = r'C:\Users\acer\Desktop\Ds\Data 2018-2023\Project\2023'

data = []

for file_name in os.listdir(folder_path):
    if file_name.endswith('.json'):
        with open(os.path.join(folder_path, file_name), 'r', encoding='utf-8') as fd:
            sc = json.load(fd)
            subject_areas = sc['abstracts-retrieval-response']['subject-areas']['subject-area']
            if isinstance(subject_areas, list):
                for subject in subject_areas:
                    data.append({
                        'file_name': file_name,
                        'fa': subject.get('@_fa', ''),
                        'subject': subject.get('$', ''),
                        'code': subject.get('@code', ''),
                        'abbrev': subject.get('@abbrev', '')
                    })
            else:
                subject = subject_areas
                data.append({
                    'file_name': file_name,
                    'fa': subject.get('@_fa', ''),
                    'subject': subject.get('$', ''),
                    'code': subject.get('@code', ''),
                    'abbrev': subject.get('@abbrev', '')
                })

subject_df = pd.DataFrame(data)

In [ ]:
subject_df

In [ ]:
def check_subjects(df, subjects):
    """
    Checks if the specified subjects exist in the DataFrame and identifies the files they are in.

    Parameters:
    - df (pd.DataFrame): DataFrame with 'file_name' and 'subject_area' columns.
    - subjects (list): List of subject names to search for.

    Returns:
    - matched_files (list): List of unique file names containing the specified subjects.
    - df_matched (pd.DataFrame): DataFrame listing file names and their matched subjects.
    """
    matched_files = []
    matched_data = []

    for subject in subjects:
        # Perform case-insensitive matching
        matches = df[df['subject'].str.lower() == subject.lower()]
        
        if not matches.empty:
            # Get unique file names where the subject exists
            files = matches['file_name'].unique().tolist()
            matched_files.extend(files)
            
            # Append matched subjects and corresponding file names
            for file in files:
                matched_data.append({'file_name': file, 'subject': subject})
        else:
            print(f"Subject '{subject}' does not exist.")

    # Remove duplicate file names
    matched_files = list(set(matched_files))

    # Create a DataFrame for matched subjects and files
    df_matched = pd.DataFrame(matched_data)

    return matched_files, df_matched

In [ ]:
subjects_to_search = ['finance','Economics and Econometrics','Economics, Econometrics and Finance (all)']
matched_files, df_matched_subjects = check_subjects(subject_df, subjects_to_search)

In [ ]:
print("Files containing the specified subjects:")
print(matched_files)

In [ ]:
df_matched_subjects

In [ ]:
ref_list = []

for file_name in matched_files:
    # Ensure full path to the file
    full_path = os.path.join(folder_path, file_name)
    
    # Check if the file exists before opening it
    if os.path.exists(full_path):
        with open(full_path, 'r', encoding='utf-8') as file:
            sc = json.load(file)

        references = sc['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference']

        for ref in references:
            ref_id = ref.get('@id', '')
            fulltext = ref.get('ref-fulltext', '')
            publication_year = ref.get('ref-info', {}).get('ref-publicationyear', {}).get('@first', '')
            title = ref.get('ref-info', {}).get('ref-title', {}).get('ref-titletext', '')
            source_title = ref.get('ref-info', {}).get('ref-sourcetitle', '')

            authors = ref.get('ref-info', {}).get('ref-authors', {}).get('author', [])
            author_names = [author.get('ce:indexed-name', '') for author in authors]
            authors_str = ', '.join(author_names)

            ref_list.append({
                'id': ref_id,
                'fulltext': fulltext,
                'year': publication_year,
                'title': title,
                'source_title': source_title,
                'authors': authors_str
            })
    else:
        print(f"File not found: {full_path}")


In [ ]:
ref_list

In [ ]:
ref_df = pd.DataFrame(ref_list)

# Replace empty strings with NaN
ref_df.replace('', np.nan, inplace=True)

ref_df.dropna(inplace=True)

# Optionally, reset the index after dropping rows
ref_df.reset_index(drop=True, inplace=True)

In [ ]:
ref_df